## Oleh:
- Putu Gde Kenzie Carlen Mataram (71230994)
<br>
- Tara Tirtanata (71231056)

# Analisis Strategi Trading Hybrid: Leveraged Pyramid pada Nasdaq-100

**Metode:** Hybrid Quantitative Trading (Machine Learning + Technical Analysis)
**Manajemen Modal:** Anti-Martingale (Pyramiding) dengan **Leverage 3x**.

Notebook ini menguji strategi algoritmik **V27: The Leveraged Pyramid (High Octane)**. Strategi ini dirancang untuk memaksimalkan profit pada indeks QQQ (Nasdaq100) dengan cara:
1.  **Filter Tren (AI):** Menggunakan Random Forest Classifier pada data 1 Jam untuk memprediksi arah pasar.
2.  **Eksekusi Agresif:** Memulai dengan posisi kecil (Scouting), lalu menumpuk posisi besar (*Aggressive Stacking*) saat tren terkonfirmasi.
3.  **Manajemen Risiko:** Menggunakan *Trailing Stop* berbasis ATR dan *Hard Exit* EMA 50 untuk mengamankan profit.

In [1]:
%pip install backtesting yfinance pandas_ta plotly scikit-learn numpy pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from backtesting import Backtest, Strategy
from datetime import datetime, timedelta
import warnings

# Mengabaikan warning agar output bersih
warnings.filterwarnings('ignore')

c:\Users\MSI\AppData\Local\Programs\Python\Python313\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

## 1. Konfigurasi Data
* **Ticker:** **QQQ** (Invesco QQQ Trust) yang merepresentasikan indeks Nasdaq-100.
* **Data Training (ML):** 2 tahun ke belakang (Data Harian/1 Jam) untuk melatih model mengenali pola tren jangka panjang.
* **Data Testing (Backtest):** 60 hari terakhir (Data 5 Menit) untuk simulasi eksekusi presisi tinggi.

In [3]:
# ==================== CONFIGURATION ====================
TICKER = 'QQQ'  # ETF NASDAQ-100

# Setup Tanggal
end_date_obj = datetime.now()
start_date_obj = end_date_obj - timedelta(days=720) # 2 Tahun data historis
START_DATE = start_date_obj.strftime('%Y-%m-%d')

# Durasi Backtest (Yahoo Finance limit untuk intraday biasanya 60 hari)
DAYS_EXECUTION = 60

## 2. Data Pipeline
Fungsi `download_data` bertugas mengambil data historis dari Yahoo Finance. Kita mengambil dua dataset terpisah:
1.  **`data_1h`**: Untuk fitur Machine Learning (Macro View).
2.  **`data_5m`**: Untuk eksekusi trading (Micro View).

In [4]:
def download_data(ticker, interval, start=None, period=None):
    try:
        if period:
             data = yf.download(ticker, period=period, interval=interval, progress=False)
        else:
             data = yf.download(ticker, start=start, interval=interval, progress=False)
        
        # Cleaning MultiIndex column issues from yfinance
        if isinstance(data.columns, pd.MultiIndex): data.columns = data.columns.droplevel(1)
        data = data.rename(columns={'Adj Close': 'Close'})
        data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
        data = data[data['Volume'] > 0] # Hapus data tanpa volume
        return data
    except Exception as e:
        print(f"Error: {e}")
        return None

print("Downloading Data...")
# Data Training (1 Hari/Jam untuk pola jangka panjang)
data_1h = download_data(TICKER, '1d', start=START_DATE) 
# Data Eksekusi (5 Menit untuk entry presisi)
data_5m = download_data(TICKER, '5m', period=f"{DAYS_EXECUTION}d")

print(f"Data 1H Shape: {data_1h.shape}")
print(f"Data 5M Shape: {data_5m.shape}")

Data 1H Shape: (493, 5)
Data 5M Shape: (4580, 5)


## 3. Feature Engineering (Indikator Teknikal)
Langkah ini menambahkan indikator teknikal ke dalam dataset.
* **EMA 50:** Filter tren utama.
* **ATR:** Pengukur volatilitas untuk Stop Loss dinamis.
* **RSI & MACD:** Indikator momentum.
* **Bollinger Bands:** Indikator volatilitas relatif (Input utama untuk ML).

In [5]:
def add_features(df, timeframe='1h'):
    df = df.copy()
    
    # 1. Trend & Volatility
    df['EMA_50'] = ta.ema(df['Close'], length=50) 
    df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)
    
    # 2. Momentum
    df['RSI'] = ta.rsi(df['Close'], length=14)
    macd = ta.macd(df['Close'])
    # Mengambil kolom MACD yang benar dari output pandas_ta
    col_macd = [c for c in macd.columns if c.startswith('MACD_')][0]
    col_sig = [c for c in macd.columns if c.startswith('MACDs_')][0]
    df['MACD'] = macd[col_macd]
    df['MACD_Signal'] = macd[col_sig]
    
    # 3. Features for ML (Bollinger Bands & Volume)
    bb = ta.bbands(df['Close'], length=20, std=2)
    col_l = [c for c in bb.columns if c.startswith('BBL')][0]
    col_u = [c for c in bb.columns if c.startswith('BBU')][0]
    
    # %B (Posisi harga relatif terhadap band)
    df['BB_PercentB'] = (df['Close'] - bb[col_l]) / (bb[col_u] - bb[col_l])
    # Bandwidth (Lebar band)
    df['BB_Width'] = (bb[col_u] - bb[col_l]) / bb[bb.columns[1]]
    
    df['Vol_Ratio'] = df['Volume'] / df['Volume'].rolling(20).mean()
    adx = ta.adx(df['High'], df['Low'], df['Close'], length=14)
    df['ADX'] = adx[adx.columns[0]]
    
    # Seasonality (Jam perdagangan)
    if timeframe == '1h':
        df['Hour'] = df.index.hour
        
    return df

print("Processing Indicators...")
data_1h = add_features(data_1h, '1h').dropna()

Processing Indicators...


## 4. Machine Learning Model (The Brain)
Kami menggunakan **Random Forest Classifier** untuk memprediksi arah tren.
* **Target:** Apakah harga penutupan candle berikutnya lebih tinggi dari saat ini? (1 = Ya, 0 = Tidak).
* **Tujuan:** Menggunakan ML sebagai filter probabilitas. Robot hanya diizinkan membeli jika ML memprediksi kondisi pasar mendukung (*Bullish Probability*).

In [6]:
print("Training Brain (ML)...")

# Target: Next Close > Current Close (Bullish)
threshold = 0.000 
data_1h['Target'] = (data_1h['Close'].shift(-1) > data_1h['Close'] * (1 + threshold)).astype(int)

# Fitur Input
features = ['BB_PercentB', 'BB_Width', 'Vol_Ratio', 'RSI', 'ADX', 'Hour']
X = data_1h[features]
y = data_1h['Target']

# Splitting & Training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
model = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_split=10, random_state=42)
model.fit(X_train, y_train)

# Prediksi pada data latih (untuk validasi sinyal)
data_1h['ML_Prediction'] = model.predict(X)

print("Mapping Signals to Execution Data...")
# Menyiapkan data 5 menit
data_5m = add_features(data_5m, '5m').dropna()

# Menyamakan Timezone
if data_5m.index.tz is None: data_5m.index = data_5m.index.tz_localize('UTC')
if data_1h.index.tz is None: data_1h.index = data_1h.index.tz_localize('UTC')

# Menggabungkan prediksi ML (Timeframe 1H) ke data eksekusi (Timeframe 5M) menggunakan Forward Fill
data_5m['ML_Signal_1H'] = data_1h['ML_Prediction'].reindex(data_5m.index, method='ffill')
data_5m.dropna(subset=['ML_Signal_1H'], inplace=True)

Training Brain (ML)...
Mapping Signals to Execution Data...


## 5. Algoritma Strategi: Leveraged Pyramid

1.  **Entry Logic (Smart Filter):** Beli hanya jika ML Bullish + Harga di atas EMA 50 + Momentum (MACD/RSI) kuat.
2.  **Pyramiding (Anti-Martingale Agresif):**
    * **Beli Awal (Scouting):** Masuk **20%** dari Equity. Risiko awal diminimalisir.
    * **Tambah Posisi (Stacking):** Jika harga naik +0.5 ATR (tren terkonfirmasi), robot membeli lagi secara masif sebesar **50%** dari Equity.
    * **Maksimal:** Hingga **8 layer** posisi bertumpuk.
3.  **Risk Management (Optimized Safety):**
    * **Trailing Stop:** Stop Loss bergerak naik mengikuti harga tertinggi dengan jarak **2.9x ATR**.
        * *Catatan:* Multiplier 2.9 dipilih sebagai titik optimal untuk memberikan ruang gerak bagi volatilitas QQQ tanpa mengorbankan profit terlalu banyak.
    * **Panic Button:** Jika harga menyentuh Stop Loss, **SEMUA** posisi dijual sekaligus.

In [7]:
class LeveragedPyramid(Strategy):
    # Parameter Strategi
    atr_multiplier = 2.9
    max_positions = 8 # Maksimal tumpukan posisi
    
    def init(self):
        self.highest_price = 0
        self.stop_price = 0
    
    def next(self):
        # Data Market Saat Ini
        price = self.data.Close[-1]
        ema_50 = self.data.EMA_50[-1]
        rsi = self.data.RSI[-1]
        macd = self.data.MACD[-1]
        macd_sig = self.data.MACD_Signal[-1]
        atr = self.data.ATR[-1]
        ml_signal = self.data.ML_Signal_1H[-1]
        
        # === EXIT LOGIC (Trailing Stop) ===
        if self.position.is_long:
            # Update Harga Tertinggi & Stop Loss
            if price > self.highest_price:
                self.highest_price = price
                new_stop = self.highest_price - (atr * self.atr_multiplier)
                self.stop_price = max(self.stop_price, new_stop)
            
            # Kena Stop Loss -> Jual Semua
            if price < self.stop_price:
                self.position.close()
                return
            
            # Trend Patah (Hard Exit)
            if price < ema_50:
                self.position.close()
                return

        # === ENTRY LOGIC (Pyramiding) ===
        # Syarat Masuk: ML Bullish, Trend Naik, Momentum Kuat
        macro = (ml_signal == 1)
        trend_ok = (price > ema_50)
        momentum_ok = (macd > macd_sig) and (rsi > 50)
        
        if macro and trend_ok and momentum_ok:
            
            # KONDISI A: Belum punya posisi (Entry Pertama)
            if not self.position:
                # Beli Agresif: 20% Equity
                self.buy(size=0.20) 
                self.highest_price = price
                self.stop_price = price - (atr * self.atr_multiplier)
            
            # KONDISI B: Sudah punya posisi (Nambah/Stacking)
            elif len(self.trades) < self.max_positions:
                # Cek harga beli terakhir
                last_entry = self.trades[-1].entry_price
                
                # Jika profit sudah > 0.5 ATR, Tambah Muatan!
                if price > last_entry + (0.5 * atr):
                    self.buy(size=0.50) # Beli lagi 50% Equity

## 6. Eksekusi Backtest (Skenario High Performance)
Simulasi dijalankan dengan parameter agresif untuk menguji ketahanan strategi:
* **Modal Awal:** $100,000
* **Komisi:** 0.0 (Simulasi Broker Zero-Fee / ETF).
* **Margin:** **0.33** (Setara **Leverage 3x**).
    * Ini mensimulasikan penggunaan instrumen *Leveraged ETF* (seperti TQQQ) atau akun margin agresif.
    * Robot diizinkan membeli aset hingga 300% dari nilai tunai akun.

In [8]:
# margin=0.5 artinya Leverage 1:2 (Bisa beli 2x modal)
bt = Backtest(data_5m, LeveragedPyramid, cash=10_000, commission=0.0003, margin=0.33) #minimal modal di 10000 dolar
stats = bt.run()

print(stats)

# Highlight Metrics Penting
print("\n--- KEY PERFORMANCE INDICATORS ---")
print(f"Win Rate: {stats['Win Rate [%]']:.2f}%")
print(f"Total Return: {stats['Return [%]']:.2f}%")
print(f"Buy & Hold Return: {stats['Buy & Hold Return [%]']:.2f}%")
print(f"Sharpe Ratio: {stats['Sharpe Ratio']:.2f}")
print(f"Max Drawdown: {stats['Max. Drawdown [%]']:.2f}%")
print(f"Profit Factor: {stats['Profit Factor']:.2f}")

Backtest.run:   0%|          | 0/4530 [00:00<?, ?bar/s]

Start                     2025-09-17 17:35...
End                       2025-12-10 18:40...
Duration                     84 days 01:05:00
Exposure Time [%]                    50.23174
Equity Final [$]                  10995.27229
Equity Peak [$]                   11737.82437
Commissions [$]                    1102.28979
Return [%]                            9.95272
Buy & Hold Return [%]                 5.75425
Return (Ann.) [%]                    52.80945
Volatility (Ann.) [%]                39.39315
CAGR [%]                             32.90814
Sharpe Ratio                          1.34057
Sortino Ratio                         4.31335
Calmar Ratio                          5.82394
Alpha [%]                             5.42872
Beta                                   0.7862
Max. Drawdown [%]                    -9.06765
Avg. Drawdown [%]                    -0.77742
Max. Drawdown Duration       27 days 23:10:00
Avg. Drawdown Duration        1 days 11:57:00
# Trades                          

In [9]:
# Menampilkan grafik ekuitas dan posisi
# Perhatikan garis hijau (Equity) yang melonjak saat Pyramiding berhasil
if stats['# Trades'] > 0:
    bt.plot()